### WE take the daily_crsp dataset and see if it has some interesting 


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

crsp = pd.read_csv("../../../data/Targets/daily_crsp.csv", )

crsp.head()

C:\Users\Filo\AppData\Local\Temp\ipykernel_2644\1432599301.py:6: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp = pd.read_csv("../../../data/Targets/daily_crsp.csv", )


,PERMNO,HdrCUSIP,CUSIP,Ticker,TradingSymbol,PERMCO,SICCD,NAICS,DlyCalDt,DlyRet,sprtrn
0,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-03,0.007353,-0.009549
1,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-04,-0.014599,-0.038345
2,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-05,0.014815,0.001922
3,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-06,-0.007299,0.000956
4,10001,36720410,29274A10,EWST,EWST,7953,4920,0,2000-01-07,-0.007353,0.027090


In [3]:
crsp["DATE"] = pd.to_datetime(crsp["DlyCalDt"])

crsp_hf = crsp[(crsp["DATE"] >= "2001-12-01") & (crsp["DATE"] <= "2001-12-31")][["PERMNO", "SICCD", "TradingSymbol"]]
crsp_hf.shape

(153067, 3)

In [4]:
crsp_hf.head()

,PERMNO,SICCD,TradingSymbol
480,10001,4920,EWST
481,10001,4920,EWST
482,10001,4920,EWST
483,10001,4920,EWST
484,10001,4920,EWST


we can use this dataset to get the permno and the industry for our hf stocks

In [5]:
crsp_features = crsp_hf[["PERMNO", "SICCD", "TradingSymbol"]].drop_duplicates()

crsp_features

,PERMNO,SICCD,TradingSymbol
480,10001,4920,EWST
4906,10002,6020,SABC
8423,10012,3670,DPAC
10165,10019,3610,IFRS
10782,10025,3070,AEPI
...,...,...,...
45869048,92807,6021,ASRV
46056087,92874,3690,VIDE
46213099,92930,3714,VOLVY
46581562,93105,6020,WBPR


In [6]:
hf = pd.read_parquet("../../../data/high_10m.parquet")

hf.head()

,DATE,SYMBOL,TIME,ALL_EX,MID_OPEN,SUM_DELTA,RETURN_NoOVERNIGHT,RETURN_SiOVERNIGHT
0,2021-12-01,A,09:30:00,DNYBTJVPCZUKXHA,152.190,-8.401819,0.000000,0.000000
1,2021-12-01,A,09:40:00,PBTJVDZUYKNHCXM,150.490,0.473357,-0.011170,-0.011170
2,2021-12-01,A,09:50:00,KDPBTJVNZUCXYH,151.475,-10.867709,0.006545,0.006545
3,2021-12-01,A,10:00:00,DABTJNKYVPZUXCH,150.510,-0.125414,-0.006371,-0.006371
4,2021-12-01,A,10:10:00,DVJTPNKUZYACBXHM,151.960,-1.274648,0.009634,0.009634


In [7]:
hf_merged = hf.merge(crsp_features, left_on= "SYMBOL", right_on="TradingSymbol", how = "left")

hf_merged.head()

,DATE,SYMBOL,TIME,ALL_EX,MID_OPEN,SUM_DELTA,RETURN_NoOVERNIGHT,RETURN_SiOVERNIGHT,PERMNO,SICCD,TradingSymbol
0,2021-12-01,A,09:30:00,DNYBTJVPCZUKXHA,152.190,-8.401819,0.000000,0.000000,NaN,NaN,NaN
1,2021-12-01,A,09:40:00,PBTJVDZUYKNHCXM,150.490,0.473357,-0.011170,-0.011170,NaN,NaN,NaN
2,2021-12-01,A,09:50:00,KDPBTJVNZUCXYH,151.475,-10.867709,0.006545,0.006545,NaN,NaN,NaN
3,2021-12-01,A,10:00:00,DABTJNKYVPZUXCH,150.510,-0.125414,-0.006371,-0.006371,NaN,NaN,NaN
4,2021-12-01,A,10:10:00,DVJTPNKUZYACBXHM,151.960,-1.274648,0.009634,0.009634,NaN,NaN,NaN


sevesoletizia 
pw: Casabianca3! 

In [10]:
import sys
from pathlib import Path

# Add the root directory (2 levels up from your notebook)
sys.path.append(str(Path.cwd().resolve().parents[2]))

# Now you can import
from utils.tool_kit import augment_features

df_final = augment_features(df = hf_merged, symbol_col = "SYMBOL", return_col="RETURN_NoOVERNIGHT", time_col= "DATE")

df_final.head()

,DATE,SYMBOL,TIME,ALL_EX,MID_OPEN,SUM_DELTA,RETURN_NoOVERNIGHT,RETURN_SiOVERNIGHT,PERMNO,SICCD,...,ma_1h,std_1h,return_cos,return_sin,return_tanh,return_exp,return_sign,return_square,return_log1p,return_relu
0,2021-12-01,A,09:30:00,DNYBTJVPCZUKXHA,152.190,-8.401819,0.000000,0.000000,NaN,NaN,...,0.000000,NaN,1.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000
1,2021-12-01,A,09:40:00,PBTJVDZUYKNHCXM,150.490,0.473357,-0.011170,-0.011170,NaN,NaN,...,-0.005585,0.007899,0.999938,-0.011170,-0.011170,0.988892,-1.0,0.000125,-0.011108,0.000000
2,2021-12-01,A,09:50:00,KDPBTJVNZUCXYH,151.475,-10.867709,0.006545,0.006545,NaN,NaN,...,-0.001542,0.008958,0.999979,0.006545,0.006545,1.006567,1.0,0.000043,0.006524,0.006545
3,2021-12-01,A,10:00:00,DABTJNKYVPZUXCH,150.510,-0.125414,-0.006371,-0.006371,NaN,NaN,...,-0.002749,0.007702,0.999980,-0.006371,-0.006371,0.993650,-1.0,0.000041,-0.006350,0.000000
4,2021-12-01,A,10:10:00,DVJTPNKUZYACBXHM,151.960,-1.274648,0.009634,0.009634,NaN,NaN,...,-0.000272,0.008670,0.999954,0.009634,0.009634,1.009680,1.0,0.000093,0.009588,0.009634


In [11]:
df_final.to_parquet("../../../data/data_transformer.parquet", index= False)